<a href="https://colab.research.google.com/github/semishen/DL-CVMarathon/blob/master/Day026_RPN_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 學習搭建RPN層

## 『本次練習目的』
  #### 了解Object Detection演算法中是如何做到分類又回歸BBOX座標

In [1]:
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed


input_shape_img = (1024, 1024, 3) # so big
img_input = Input(shape=input_shape_img)

# CNN to get feature maps
def nn_base(img_input):
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)# (1024,1024,64) 
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x) # (1024,1024,64) 
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x) # (512,512,64)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)# (512,512,128)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)# (512,512,128)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x) #(256,256,128)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)#(256,256,256)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)#(256,256,256)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)#(256,256,256)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)#(128,128,256)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)#(128,128,512)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)#(128,128,512)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)#(128,128,512)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)#(64,64,512)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)#(64,64,512)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)#(64,64,512)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)#(64,64,512)

    return x

Using TensorFlow backend.


In [2]:
# RPN
def rpn(base_layers, num_anchors):

    x = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer='normal', name='rpn_conv1')(base_layers)

    # rpn分類和迴歸
    x_class = Conv2D(num_anchors*2, (1, 1), activation='softmax',name='rpn_out_class')(x)
    x_reg = Conv2D(num_anchors *4, (1, 1), activation='linear', name='rpn_out_regress')(x)

    return x_class, x_reg, base_layers

In [3]:
base_layers=nn_base(img_input)

In [4]:
x_class, x_reg, base_layers=rpn(base_layers, 9)

In [5]:
print('Classification支線：',x_class) #'''確認深度是否為18'''
print('BBOX Regression 支線：',x_reg) #'''確認深度是否為36'''
print('CNN Output：',base_layers)

Classification支線： Tensor("rpn_out_class/truediv:0", shape=(None, 64, 64, 18), dtype=float32)
BBOX Regression 支線： Tensor("rpn_out_regress/BiasAdd:0", shape=(None, 64, 64, 36), dtype=float32)
CNN Output： Tensor("block5_conv3/Relu:0", shape=(None, 64, 64, 512), dtype=float32)
